In [231]:
import random
import math
import copy

from collections import defaultdict
def isSorted(nodes):
    if(len(nodes)==0):
        return True
    for i in range(1,len(nodes)):
        if(nodes[i-1] > nodes[i]):
            return False
    return True

def EuclideanDistance(point1_x, point1_y, point2_x, point2_y):
    internalCalculation = math.pow((point1_x - point2_x),2) + math.pow((point1_y - point2_y),2)
    return math.sqrt(internalCalculation);

#GEOM-norm
M_PI = 3.14159265358979323846264

def geom_edgelen(xi, xj, yi, yj):
    lati = M_PI * xi / 180.0;
    latj = M_PI * xj / 180.0;

    longi = M_PI * yi / 180.0;
    longj = M_PI * yj / 180.0;

    q1 = math.cos (latj) * math.sin(longi - longj);
    q3 = math.sin((longi - longj)/2.0);
    q4 = math.cos((longi - longj)/2.0);
    q2 = math.sin(lati + latj) * q3 * q3 - math.sin(lati - latj) * q4 * q4;
    q5 = math.cos(lati - latj) * q4 * q4 - math.cos(lati + latj) * q3 * q3;
    return (int) (6378388.0 * math.atan2(sqrt(q1*q1 + q2*q2), q5) + 1.0);

def getDistanceBetweenNodes(node, otherNode, AdjHash, Map, DistanceComputation):
    if(node == otherNode):
        return 0;
    if(otherNode in AdjHash and node in AdjHash[otherNode]):
        return AdjHash[otherNode][node]
    return DistanceComputation(Map[node][0], Map[node][1], Map[otherNode][0], Map[otherNode][1])

def computeAdjHashOfCompleteGraph(Map, DistanceComputation):
    # Input::  Map: {0: (0.13037375991631983, 0.17982980099790336), ... }
    nodes = Map.keys()
    assert isSorted(nodes)
    AdjHash= {}
    for node in nodes:
        AdjHash[node] = {}
        for otherNode in nodes:
            AdjHash[node][otherNode] = getDistanceBetweenNodes(node, otherNode, AdjHash, Map, DistanceComputation)
    return AdjHash

def TspGenerator(numberOfCities, lowRange=0.0, highRange=1.0, DistanceComputation=EuclideanDistance):
    Map = {}
    inverseMap = {}
    AdjHash = {}
    for x in range(0, numberOfCities):
        coordinate = (random.uniform(lowRange, highRange), random.uniform(lowRange, highRange))
        tries = 0;
        while(coordinate in inverseMap):
            coordinate = (random.uniform(lowRange, highRange), random.uniform(lowRange, highRange))
            tries+=1;
            if(tries==5):
                print "Unable to Generate Coordinates"
                return ;
        Map[x] = coordinate
        inverseMap[coordinate] = x
    return Map, inverseMap, computeAdjHashOfCompleteGraph(Map, DistanceComputation);



In [271]:
def isAllVisited(VisitedHash, nodes):
    for node in nodes:
        if(node not in VisitedHash):
            return False
    return True

def PrimsAlgorithm(startNode, nodes, AdjHash):
    #Input startNode : type int, nodes : type list of int's
    MSTCost = 0;
    h = []
    visitedHash = {}
    visitedHash[startNode] = True
    prevNode = startNode
    MstEdges = []
    while(not isAllVisited(visitedHash, nodes)):
        for node in nodes:
            if(node not in visitedHash.keys()):
                heappush(h, (AdjHash[prevNode][node], (prevNode, node)))
        cost, edge = heappop(h)
        parentNode, minNode = edge
        while(minNode in visitedHash):
            # if min node was alreadyVisited
            cost, edge = heappop(h)
            parentNode, minNode = edge
        MSTCost += cost
        MstEdges.append((parentNode, minNode))
        visitedHash[minNode] = True
    return MSTCost, MstEdges;

def MSTHeuristic(startNode, nodes, AdjHash):
    #Input :: AdjHash is a Adjacency map of the entire set of nodes with the value being the distance
    assert startNode in nodes;
    PrimsAlgorithm(startNode, nodes, AdjHash)
    
    
class TspNode:
    def __init__(self, x, y, nodeNumber, path_cost, heuristicCost, parentNode):
        self.pos_x = x
        self.pos_y = y
        self.node_number = nodeNumber
        self.h_cost = heuristicCost
        self.g_cost = path_cost
        self.f_cost = self.g_cost + self.h_cost
        self.parent = parentNode

def goalTest(node, listOfNodeNumbers):
    return isHamiltonianCycle(node, listOfNodeNumbers)

def appendToFrontier(newNode, Frontier):
    if(newNode.f_cost in Frontier):
        Frontier[newNode.f_cost].append(newNode)
    else:
        Frontier[newNode.f_cost] = [newNode]

def printPathTraversed(node):
    pathTraversed = []
    while(node.parent!=-1):
        pathTraversed.append(node.node_number)
        node=node.parent
    pathTraversed.append(node.node_number)
    return pathTraversed
    
def isHamiltonianCycle(node, listOfNodeNumbers):
    pathTraversed = [node.node_number]
    visited = {}
    while(node.parent!=-1):
        pathTraversed.append(node.node_number)
        visited[node.node_number] = True
        node = node.parent
    pathTraversed.append(node.node_number)
    visited[node.node_number] = True
    # Check if ALL NODES are visited
    for nodeNumber in listOfNodeNumbers:
        if nodeNumber not in visited:
            return False
    # Check if cycle is present
    if(pathTraversed[0] == pathTraversed[len(pathTraversed)-1]):
        return True
    return False;

def remove_empty_keys(d):
    for k in d.keys():
        if not d[k]:
            del d[k]

def popMinNode(Frontier):
#     remove_empty_keys(Frontier)
#     if(len(Frontier)==0):
#         return None
    m = min(i for i in Frontier.keys() if len(Frontier[i]) > 0)
    minNode = Frontier[m].pop(0)
    return minNode;
    
def edge(node):
    if(node.parent==-1):
        pNumber = -1
    else:
        pNumber = node.parent.node_number
    return (pNumber, node.node_number)
    
def hasTraversedPreviously(newEdge, bestNode):
    if(newEdge in bestNode.edges_Traversed):
        return True
    return False

def findBestNode(Frontier):
    bestNode = popMinNode(Frontier)
#     if(bestNode == None):
#         return None
#     print bestNode.edges_Traversed
#     print edge(bestNode)
    while((hasTraversedPreviously(edge(bestNode), bestNode))):
        bestNode = popMinNode(Frontier)
#         if(bestNode == None):
#             return None
    bestNode.edges_Traversed[edge(bestNode)] = True
    return bestNode
        
def checkIfFCostIsGreaterThanBestFinalNode(bestFinalNode, node):
    if(bestFinalNode.f_cost <= node.f_cost):
        return False
    return True

def checkNoOtherBetterNodeToExpand(Frontier, finalNode):
    bestNodeToExpand = findBestNode(Frontier)
    return checkIfFCostIsGreaterThanBestFinalNode(finalNode, bestNodeToExpand)

def appendToFrontier(newNode, Frontier):
    if(newNode.f_cost in Frontier):
        Frontier[newNode.f_cost].append(newNode)
    else:
        Frontier[newNode.f_cost] = [newNode]

def expandNode(Node, Frontier, AdjHash, listOfNodeNumbers, Map, heuristicCost):
#     print listOfNodeNumbers
    for nodeNumber in listOfNodeNumbers:
        if(nodeNumber != Node.node_number):
            latitude = Map[nodeNumber][0]
            longitude = Map[nodeNumber][1]
#             print "Expanding node number %d %d" %(Node.node_number, nodeNumber)
            tspNode = TspNode(latitude, longitude, nodeNumber, Node.g_cost+AdjHash[Node.node_number][nodeNumber], heuristicCost, Node)
            tspNode.edges_Traversed = copy.copy(Node.edges_Traversed)
            appendToFrontier(tspNode, Frontier)
#             print Frontier
    return ;
    
def solveAstar(startNode, AdjHash, listOfNodeNumbers, Map, heuristicCost):
    #Input startNode : type int, listOfNodeNumbers : type list of int's
    latitude = Map[startNode][0]
    longitude = Map[startNode][1]
    sNode = TspNode(latitude, longitude, startNode, 0, heuristicCost, -1)
    sNode.edges_Traversed = {}
    Frontier = defaultdict(list)
    Frontier[sNode.f_cost] = [sNode]
    isCompleted = False
    bestFinalNode = None
    FoundCostToFinal = False;
    noOfNodesExpanded = 0
    VisitedEdgeHashSet = {}
    while(len(Frontier)!=0 or isCompleted):
        bestNode = findBestNode(Frontier)
#         if(bestNode==None):
#             break;
        if(bestNode.g_cost <= heuristicCost):
            isNodeGoalState = goalTest(bestNode, listOfNodeNumbers)
            if(FoundCostToFinal):
                if(not checkIfFCostIsGreaterThanBestFinalNode(bestFinalNode, bestNode)):
                    break
            if(isNodeGoalState):
                if(not FoundCostToFinal):
                    bestFinalNode = bestNode
                    FoundCostToFinal = True
                else:
                    if(bestFinalNode.g_cost > bestNode.g_cost):
                        bestFinalNode=bestNode
            if(isNodeGoalState and checkNoOtherBetterNodeToExpand(Frontier, bestNode)):
                isCompleted = True
                break
            visitingEdge = edge(bestNode)
#             print "Expanding Node Number %d with co-ordinates (%s,%s) and visiting edge (%d, %d) and " %(bestNode.node_number, bestNode.pos_x, bestNode.pos_y, visitingEdge[0], visitingEdge[1])
#             print printPathTraversed(bestNode), bestNode.g_cost, bestNode.f_cost
            noOfNodesExpanded+=1
#             print Frontier
            expandNode(bestNode, Frontier, AdjHash, listOfNodeNumbers, Map, heuristicCost)
#             VisitedEdgeHashSet[visitingEdge] = True
            
        else:
            break
    return bestFinalNode, noOfNodesExpanded

In [233]:
Map, inverseMap, AdjHash = TspGenerator(5)
nodes = Map.keys()

In [234]:
heuristicCost, EdgesMst = PrimsAlgorithm(0, nodes, AdjHash)
heuristicCost *= 2;

In [235]:
startNode = 0
bestFinalNode, noOfNodesExpanded = solveAstar(startNode, AdjHash, nodes, Map, heuristicCost)

Expanding Node Number 0 with co-ordinates (0.569484830972,0.0733476693194) and visiting edge (-1, 0) and 
[0] 0 3.84096940338
Expanding Node Number 3 with co-ordinates (0.70362153698,0.146721277248) and visiting edge (0, 3) and 
[3, 0] 0.152893238042 3.99386264142
Expanding Node Number 1 with co-ordinates (0.69358214858,0.347301650669) and visiting edge (0, 1) and 
[1, 0] 0.300750607871 4.14172001125
Expanding Node Number 0 with co-ordinates (0.569484830972,0.0733476693194) and visiting edge (3, 0) and 
[0, 3, 0] 0.305786476084 4.14675587946
Expanding Node Number 1 with co-ordinates (0.69358214858,0.347301650669) and visiting edge (3, 1) and 
[1, 3, 0] 0.353724698529 4.1946941019
Expanding Node Number 4 with co-ordinates (0.615390869954,0.558721049647) and visiting edge (0, 4) and 
[4, 0] 0.487539416607 4.32850881998
Expanding Node Number 3 with co-ordinates (0.70362153698,0.146721277248) and visiting edge (1, 3) and 
[3, 1, 0] 0.501582068359 4.34255147173
Expanding Node Number 4 with 

In [236]:
printPathTraversed(bestFinalNode)

[0, 2, 4, 1, 3, 0]

In [237]:
# Testing Prims Algorithm
ExamplestartNode = 1
Examplenodes = [1,2,3,4]
ExampleAdjHash =  {
    1:{ 
        1: 0,
        2: 10,
        3: 15,
        4: 20
      },
    2:{ 
        1: 10,
        2: 0,
        3: 350,
        4: 250
      },
    3:{ 
        1: 15,
        2: 350,
        3: 0,
        4: 300
      },
    4:{ 
        1: 20,
        2: 250,
        3: 300,
        4: 0
      }
}
ExampleMap = {
    1:(0,0),
    2:(1,1),
    3:(2,2),
    4:(3,3)
}
ExampleheuristicCost, Edges = PrimsAlgorithm(ExamplestartNode, Examplenodes, ExampleAdjHash)
ExampleheuristicCost *= 2

In [238]:
ExampleheuristicCost

90

In [239]:
bestNode, noOfNodesExpanded = solveAstar(ExamplestartNode, ExampleAdjHash, Examplenodes, ExampleMap, ExampleheuristicCost)

Expanding Node Number 1 with co-ordinates (0,0) and visiting edge (-1, 1) and 
[1] 0 90
Expanding Node Number 2 with co-ordinates (1,1) and visiting edge (1, 2) and 
[2, 1] 10 100
Expanding Node Number 3 with co-ordinates (2,2) and visiting edge (1, 3) and 
[3, 1] 15 105
Expanding Node Number 4 with co-ordinates (3,3) and visiting edge (1, 4) and 
[4, 1] 20 110
Expanding Node Number 1 with co-ordinates (0,0) and visiting edge (2, 1) and 
[1, 2, 1] 20 110
Expanding Node Number 1 with co-ordinates (0,0) and visiting edge (3, 1) and 
[1, 3, 1] 30 120
Expanding Node Number 3 with co-ordinates (2,2) and visiting edge (1, 3) and 
[3, 1, 2, 1] 35 125
Expanding Node Number 1 with co-ordinates (0,0) and visiting edge (4, 1) and 
[1, 4, 1] 40 130
Expanding Node Number 4 with co-ordinates (3,3) and visiting edge (1, 4) and 
[4, 1, 2, 1] 40 130
Expanding Node Number 2 with co-ordinates (1,1) and visiting edge (1, 2) and 
[2, 1, 3, 1] 40 130
Expanding Node Number 4 with co-ordinates (3,3) and visit

In [240]:
print printPathTraversed(bestNode)
print noOfNodesExpanded, bestNode.g_cost

[1, 4, 1, 3, 1, 2, 1]
26 90


In [267]:
def getInputFromFile(fileName):
    try:
        Map = {}
        inverseMap = {}
        in_file = open(fileName, 'r')
        lines = in_file.readlines()
        for line in lines[7:len(lines)-1]:
            data = line.split()
            nodeNo = int(data[0])
            latitude, longitude = float(data[1]), float(data[2])
            coordinate = (latitude, longitude)
            Map[nodeNo] = coordinate
            inverseMap[coordinate] = nodeNo
    finally:
        in_file.close()
    return Map, inverseMap, computeAdjHashOfCompleteGraph(Map, DistanceComputation=EuclideanDistance)


In [274]:
WCityMap, WCityinverseMap, WCityAdjHash = getInputFromFile('wi29.tsp.txt')
WCitystartNode = 1
WCitynodes = WCityMap.keys()
WCityheuristicCost, WCityEdges = PrimsAlgorithm(WCitystartNode, WCitynodes, WCityAdjHash)
WCityheuristicCost *= 2


In [276]:
WCityheuristicCost
WCityMap

{1: (20833.3333, 17100.0),
 2: (20900.0, 17066.6667),
 3: (21300.0, 13016.6667),
 4: (21600.0, 14150.0),
 5: (21600.0, 14966.6667),
 6: (21600.0, 16500.0),
 7: (22183.3333, 13133.3333),
 8: (22583.3333, 14300.0),
 9: (22683.3333, 12716.6667),
 10: (23616.6667, 15866.6667),
 11: (23700.0, 15933.3333),
 12: (23883.3333, 14533.3333),
 13: (24166.6667, 13250.0),
 14: (25149.1667, 12365.8333),
 15: (26133.3333, 14500.0),
 16: (26150.0, 10550.0),
 17: (26283.3333, 12766.6667),
 18: (26433.3333, 13433.3333),
 19: (26550.0, 13850.0),
 20: (26733.3333, 11683.3333),
 21: (27026.1111, 13051.9444),
 22: (27096.1111, 13415.8333),
 23: (27153.6111, 13203.3333),
 24: (27166.6667, 9833.3333),
 25: (27233.3333, 10450.0),
 26: (27233.3333, 11783.3333),
 27: (27266.6667, 10383.3333),
 28: (27433.3333, 12400.0),
 29: (27462.5, 12992.2222)}

In [272]:
WCitybestNode, WCitynoOfNodesExpanded = solveAstar(WCitystartNode, WCityAdjHash, WCitynodes, WCityMap, WCityheuristicCost)

KeyboardInterrupt: 